In [28]:
import psycopg2
from psycopg2.extras import execute_values

# Establish a connection to the database
# Adjust the connection parameters as necessary
conn = psycopg2.connect(
    dbname="twitty",
    user="postgresuser",
    password="postgrespassword",
    host="localhost",
    port="5432"
)


In [29]:
cur = conn.cursor()

In [3]:
!pip freeze | grep numpy

numpy @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_97k4h5q73m/croot/numpy_and_numpy_base_1704311708477/work/dist/numpy-1.26.3-cp310-cp310-macosx_10_9_x86_64.whl#sha256=9d4230f09aa6cd18b645548b55fa74cb611df850cd99d39a1b636402e018c785


In [77]:
import pandas as pd
import numpy as np

In [124]:
twitter_df = pd.read_csv('/Users/nofary/IDC/BDP/twitter-bdp/src/data_digest/tweets_df.csv', delimiter='	')
# tweets_paritioned_by_word = pd.read_csv('/Users/nofary/IDC/BDP/twitter-bdp/src/data_digest/tweets_paritioned_by_word.csv', delimiter='	')


In [118]:
twitter_df = twitter_df.replace({np.nan: None})
twitter_df.drop(columns='Unnamed: 0',inplace=True)




In [119]:
tweets_paritioned_by_word = twitter_df.explode('parsed_content')
tweets_paritioned_by_word = tweets_paritioned_by_word.dropna(subset=["parsed_content"])

In [120]:
import ast

def convert_to_list(row):
    try:
        return ast.literal_eval(row)
    except ValueError:
        return []


twitter_df['parsed_content'] = twitter_df['parsed_content'].apply(convert_to_list)

raw_twitter_df = list(twitter_df[['tweet_id', 'author', 'content', 'country', 'date_time', 'language', 'latitude', 'longitude', 'number_of_likes', 'number_of_shares', 'parsed_content']].itertuples(index=False, name=None))


In [109]:

# tweets_by_likes
insert_tweets_by_likes = """INSERT INTO tweets_by_likes (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
execute_values(cur, insert_tweets_by_likes, raw_twitter_df)

# tweets_by_share
insert_tweets_by_share = """INSERT INTO tweets_by_share (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
execute_values(cur, insert_tweets_by_share, raw_twitter_df)

# user_tweets
insert_user_tweets = """INSERT INTO user_tweets (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
execute_values(cur, insert_user_tweets, raw_twitter_df)



In [121]:
raw_tweets_paritioned_by_word = list(tweets_paritioned_by_word[['tweet_id', 'author', 'content', 'country', 'date_time', 'language', 'latitude', 'longitude', 'number_of_likes', 'number_of_shares', 'parsed_content']].itertuples(index=False, name=None))


In [122]:
# user_tweets
insert_tweets_by_word = """INSERT INTO tweets_by_word (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
execute_values(cur, insert_tweets_by_word, raw_tweets_paritioned_by_word)

In [33]:
    # SQL command to delete all data from the table
    delete_query = "DELETE FROM tweets_by_likes;"

    # Execute the SQL command
    cur.execute(delete_query)

    # conn.rollback()

In [11]:
# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [23]:
import os
import pandas as pd
import numpy as np
import ast

In [24]:

def convert_to_list(row):
    try:
        return ast.literal_eval(row)
    except ValueError:
        return []

In [25]:
def digest_data():
    conn = None
    cur = None
    try:
        # Database connection parameters from environment variables
        # dbname = os.getenv("POSTGRES_DB", "twitty")
        user = os.getenv("POSTGRES_USER", "postgresuser")
        password = os.getenv("POSTGRES_PASSWORD", "postgrespassword")
        host = os.getenv("DB_URL", "localhost")
        port = os.getenv("DB_PORT", "5432")

        conn = psycopg2.connect(
            dbname='twitty',
            user=user,
            password=password,
            host=host
            # port=port
        )
        cur = conn.cursor()

        # Load and preprocess data
        twitter_df = pd.read_csv('/Users/nofary/IDC/BDP/twitter-bdp/src/data_digest/tweets_df.csv', delimiter='	')
        twitter_df = twitter_df.replace({np.nan: None})

        tweets_paritioned_by_word = twitter_df.explode('parsed_content')
        tweets_paritioned_by_word = tweets_paritioned_by_word.dropna(subset=["parsed_content"])

        twitter_df['parsed_content'] = twitter_df['parsed_content'].apply(convert_to_list)

        raw_twitter_df = list(twitter_df[['tweet_id', 'author', 'content', 'country', 'date_time', 'language', 'latitude', 'longitude', 'number_of_likes', 'number_of_shares', 'parsed_content']].itertuples(index=False, name=None))

        # tweets_by_likes
        insert_tweets_by_likes = """INSERT INTO tweets_by_likes (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
        execute_values(cur, insert_tweets_by_likes, raw_twitter_df)

        # tweets_by_share
        insert_tweets_by_share = """INSERT INTO tweets_by_share (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
        execute_values(cur, insert_tweets_by_share, raw_twitter_df)

        # user_tweets
        insert_user_tweets = """INSERT INTO user_tweets (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
        execute_values(cur, insert_user_tweets, raw_twitter_df)

        # user_tweets
        raw_tweets_paritioned_by_word = list(tweets_paritioned_by_word[['tweet_id', 'author', 'content', 'country', 'date_time', 'language', 'latitude', 'longitude', 'number_of_likes', 'number_of_shares', 'parsed_content']].itertuples(index=False, name=None))
        insert_tweets_by_word = """INSERT INTO tweets_by_word (tweet_id, author, content, country, date_time, language, latitude, longitude, number_of_likes, number_of_shares, parsed_content) VALUES %s;"""
        execute_values(cur, insert_tweets_by_word, raw_tweets_paritioned_by_word)

        # Commit the transaction
        conn.commit()

    except Exception as e:
        # Rollback the transaction in case of error
        if conn is not None:
            conn.rollback()
        print(f"An error occurred: {e}")
        # Optionally, re-raise the exception if you want the error to propagate
        raise

    finally:
        # Close the cursor and connection
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()


In [26]:
digest_data()